# Logging with R

Logging is one of the most useful tools in a code writer's toolkit. All too often, people get in the habit of using `print` statements to debug their code. While convenient, this causes a few problems. Firstly, what happens when you are done debugging? You now have to go back and remove all these `print` statements. If you forget one, you could be exposing potentially sensitive information or just generally cluttering up your user's screen with unnecesssary information. 


In this scenario, you will learn how to use R's native logging features to:

* Debug your code
* Log failed and successful execution
* Send logs to different destination

Let's get started!

## How Logging Works
A logger is basically an enhanced print statement, with a few key enhancements:

1. You can globally control when it fires. 
2. You can automatically add additional metadata such as process information, execution time etc. 
3. You can easily direct the output to different places, such as files, email, or remote log aggregators.


## The Logging Module
R does not have a logging module built into the language so is necessary to install any additional modules to run the examples in this lab.  While there are many logging modules available for R, I recommend the `logger` package which is available here: https://github.com/daroczig/logger. 

To install this package, simply execute the following command:

```R
install.packages('logger')
```

This module must be imported at the beginning of your script or notebook with the command:

In [2]:
library(logger)

## Logging Levels
R-logger has five standard logging levels which are:

* `TRACE` (0)
* `DEBUG` (10)
* `INFO` (20)
* `WARNING` (30)
* `ERROR` (40)
* `FATAL` (50)

The logging levels are heirarchical, meaning that if you set the logging level to `ERROR` you will receive any messages at the `ERROR` or `FATAL` level.  By setting the level appropriately, you can control which level of messages are displayed. In general, when you are debugging you will want to set the logging level as low as possible and when you deploy your code, you will set the logging level higher.

Note: Logging messages at the `ERROR` level do not halt execution. To do that you still have to throw an exception. 

To set the logging level, use the following command: 
```R
log_threshold(DEBUG)
```
Once you've done that, the next step is to actually add logging messages to your code. Instead of `print()` statements for debugging, you can use:
                                                      
```R
log_debug('The value of x is: {x}')
```
Note the use of `glue` formatting in the log messages.

In the example below, run the code with the logging level set to `DEBUG` then try it again with the level set to `ERROR` and watch what happens.  

In [3]:
# Set the logging level to DEBUG
log_threshold(DEBUG)

countVowels <- function(word) {
    log_info("In count vowels function")
    vowel_count <- 0
    
    letters <- strsplit(word, "")[[1]]
    for (letter in letters) {
        if (letter=='a' || letter =='e' || letter == 'i' || letter == 'o' || letter == 'u') {
            vowel_count <- vowel_count + 1
            log_info("Found a vowel: {letter}")
        } else if (letter == 'y') {
            log_info("Found a y and that is sometimes a vowel.")
        } else if (is.numeric(letter)) {
            log_error("Numbers aren't vowels!!")
        } else {
            log_warn("{letter} is not a vowel")
        }
    }
    
    return(vowel_count)
}

count <- countVowels("encylopedia")
log_info("There are {count} vowels in encylopedia")

count <- countVowels("faceb00k")    
log_info("There are {count} vowels in faceb00k")


INFO [2020-06-22 01:12:16] In count vowels function
INFO [2020-06-22 01:12:16] Found a vowel: e
WARN [2020-06-22 01:12:16] n is not a vowel
WARN [2020-06-22 01:12:16] c is not a vowel
INFO [2020-06-22 01:12:16] Found a y and that is sometimes a vowel.
WARN [2020-06-22 01:12:16] l is not a vowel
INFO [2020-06-22 01:12:16] Found a vowel: o
WARN [2020-06-22 01:12:16] p is not a vowel
INFO [2020-06-22 01:12:16] Found a vowel: e
WARN [2020-06-22 01:12:16] d is not a vowel
INFO [2020-06-22 01:12:16] Found a vowel: i
INFO [2020-06-22 01:12:16] Found a vowel: a
INFO [2020-06-22 01:12:16] There are 5 vowels in encylopedia
INFO [2020-06-22 01:12:16] In count vowels function
WARN [2020-06-22 01:12:16] f is not a vowel
INFO [2020-06-22 01:12:16] Found a vowel: a
WARN [2020-06-22 01:12:16] c is not a vowel
INFO [2020-06-22 01:12:16] Found a vowel: e
WARN [2020-06-22 01:12:16] b is not a vowel
WARN [2020-06-22 01:12:16] 0 is not a vowel
WARN [2020-06-22 01:12:16] 0 is not a vowel
WARN [2020-06-22 01

# Log Formatting
Unlike Python, log messages in R are nicely formatted by default and you probably will not have to alter the message formatting. 

However, in the event that you do wish to change the log formatting, the first step is to view what variables are available to you.  You can view the available variables with the following command. [1]

```R
?get_logger_meta_variables
```

Once you know which variables you want to include or exclude in your log messages, there are two steps:
1. First call the `layout_glue_generator()` function and create a new logger format string as shown below.
2. Next, call the `log_layout()` method to assign the new formatter to the logger.

```R
logger <- layout_glue_generator(format = '{node}/{pid}/{namespace}/{fn} {time} {level}: {msg}')
log_layout(logger)
```

You can format logs in other formats such as `json` however, I would recommend sticking with a standard logging format such as syslog. 


[1]: https://daroczig.github.io/logger/articles/customize_logger.html#log-message-formatter-functions

In [10]:
logger <- layout_glue_generator(format = '{time} {level} {pid}/{namespace}/{fn}: {msg}')
log_layout(logger)

In [11]:
log_debug("Test message")

2020-06-22 01:30:59 DEBUG 61238/global/eval: Test message


# Sending Logs to Different Destinations
Our final task is learning how to send logs to different destinations. In production situations, you may want to be notified immediately via email or SMS message if your code is throwing a `CRITICAL` but you might only want confirmation of successful execution recorded in a file.  

Let's consider the following pseudo code:

```
db_connection <- connect_to_database()
if db_connection is null:
    email_logger.critical("Database connection failed.")
    exit(1)
else:
    file_logger.info("Database connection succeeded")

# Process data...

file_logger.info("Sucessfully processed %d records", record_count)

```
This code will send an email and stop execution if the connection to the database fails. If the connection to the database succeeds, execution will continue and the script will record its successful completion in a logfile. 

## Log Namespaces
So far, we've been using the root logger. In practice, you may want to create multiple loggers to do different things with log messages. As we saw in the example above, we might want to be notified immediately via email or SMS if there is a critical failure, but we might also simply want to record successful completion of execution in a log file. 

In order to do this, you will need to define `namespaces` within the logger.  You can think of this as creating separate loggers for different purposes. The logger module allows you to do this simply by defining the `namespace` argument in any of the logging functions. 


In [14]:
log_threshold(FATAL, namespace = 'SMS_Logger')
log_threshold(DEBUG, namespace = 'debug_logger')

log_info("You should see me", namespace='debug_logger')
log_info("But not me...", namespace = "SMS_Logger")
log_fatal("Oh No!! Something serious happened")

2020-06-22 09:56:09 INFO 61238/debug_logger/eval: You should see me
2020-06-22 09:56:09 FATAL 61238/global/eval: Oh No!! Something serious happened


## Redirecting Log Output
So far, we've seen loggers that send output to the console via standard output. As you saw in the pseudo code earlier, quite often you will want to append routine messages to a file for examination later, and send more critical messages to email or even SMS. 

Obviously, as your code matures and is placed into a production state, you'll want to disable `DEBUG` messages entirely. 

In any event, what you will need to add an `appender` to your log namespace which will redirect the messages to the appropriate destination. Some example appenders include `appender_file` which sends logs to a file and `appender_slack` which sends messages to a Slack channel.

For a complete list of appenders look here: [1].  If you're interested in writing your own custom appender look here: [2].

The code below from the documentation[3] demonstrates how to use the slack appender to send log messages to Slack.

```R
## load Slack configuration, API token etc from a (hopefully encrypted) yaml file or similar
slack_config <- config::config(...)
## redirect log messages to Slack
log_appender(appender_slack(
    channel   = '#gergely-test',
    username  = 'logger',
    api_token = slack_config$token
), namespace = 'slack')
log_info('Excited about sending logs to Slack!')
#> INFO [2018-11-28 00:21:13] Excited about sending logs to Slack!
log_info('Hi there from logger@R!', namespace = 'slack')
```


[1]:https://rdrr.io/cran/logger/man/appender_file.html
[2]: https://daroczig.github.io/logger/articles/write_custom_extensions.html#custom-log-record-appenders-1
[3]: https://daroczig.github.io/logger/articles/customize_logger.html#what-gets-logged

# Conclusion 
In conclusion, in this module you have learned how to use logging to track the actions of your code and report them to different locations. I hope you see the value of logging as a replacement for print statements in debugging, as a way to track the execution of your code and send alerts when things go awry.


This scene is only a brief intro to the world of logging with R and there are many additional facets you can explore, however at a minimum, I hope I've convinced you to stop using `print()` statements and replace them with `log_debug()`. 

Finally, as a security professional I need to include the admonition to never include credentials or other sensitive information in logs!


### Further References
* https://daroczig.github.io/logger/index.html
* http://mazamascience.com/WorkingWithData/?p=1814